<a href="https://colab.research.google.com/github/Pdugovich/DS-Unit-2-Applied-Modeling/blob/master/module1/assignment_applied_modeling_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [ ] Choose your target. Which column in your tabular dataset will you predict?
- [ ] Choose which observations you will use to train, validate, and test your model. And which observations, if any, to exclude.
- [ ] Determine whether your problem is regression or classification.
- [ ] Choose your evaluation metric.
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" information from the future?


In [0]:
# Import libraries
import pandas as pd
import os

In [0]:
df_original = pd.read_csv('oh_columbus_2019_08_13.csv')

In [0]:
df_original.head()

,raw_row_number,date,time,location,lat,lng,precinct,zone,subject_race,subject_sex,type,arrest_made,citation_issued,warning_issued,outcome,search_conducted,reason_for_stop,raw_enforcement_taken
0,1|2|3,2012-01-01,01:16:00,livingston and carpenter,39.949920,-82.975067,11.0,5.0,white,female,vehicular,True,False,False,arrest,True,Moving Violation,Arrest|Vehicle Search|Driver Search
1,4,2012-01-01,01:20:00,E 16th Ave and Waldeck Ave,40.001043,-83.006871,4.0,4.0,white,male,vehicular,False,False,True,warning,False,Moving Violation,Verbal Warning
2,5,2012-01-01,07:54:00,king ave and cannon dr,39.990708,-83.020528,NaN,NaN,black,male,vehicular,False,False,True,warning,False,Moving Violation,Verbal Warning
3,6,2012-01-01,10:00:00,James rd and Livingston ave,39.946876,-82.915567,9.0,2.0,black,male,vehicular,False,False,True,warning,False,Moving Violation,Verbal Warning
4,7,2012-01-01,10:50:00,N Wilson Road and Twin Creeks,39.979569,-83.102749,15.0,3.0,black,female,vehicular,False,False,True,warning,False,Moving Violation,Verbal Warning


In [0]:
#Making a duplicate of the dataframe to retain the original
df = df_original.copy()

## Selecting a target

I'll be trying to predict whether an action is taken against the person being pulled over, this includes 'arrest' or 'citation'. The alternative is 'warning'

## Selecting how to train/test split

The dates range from the start of 2012 to the end of 2016. I'll try a couple different splits, but first, I'm going to train on 2012-2015, validate on 2015, then test on 2016. I might also try randomized or time-series crossvalidation.

In [0]:

df.dtypes

raw_row_number            object
date                      object
time                      object
location                  object
lat                      float64
lng                      float64
precinct                 float64
zone                     float64
subject_race              object
subject_sex               object
type                      object
arrest_made                 bool
citation_issued             bool
warning_issued              bool
outcome                   object
search_conducted            bool
reason_for_stop           object
raw_enforcement_taken     object
dtype: object

In [0]:
# Converting date and time to datetime format to split the data

In [0]:
df['date'] = pd.to_datetime(df['date'])

In [0]:
df['date']

0        2012-01-01
1        2012-01-01
2        2012-01-01
3        2012-01-01
4        2012-01-01
            ...    
128152   2016-12-30
128153   2016-12-30
128154   2016-12-30
128155   2016-12-30
128156   2016-12-30
Name: date, Length: 128157, dtype: datetime64[ns]

In [0]:
#Separating my data
my_train = df[df['date'] <= '2014-12-31']
my_val = df[(df['date'] >= '2015-01-01') & (df['date'] <= '2015-12-31')]
test = df[df['date'] >= '2016-01-01']

In [0]:
my_train.head()

,raw_row_number,date,time,location,lat,lng,precinct,zone,subject_race,subject_sex,type,arrest_made,citation_issued,warning_issued,outcome,search_conducted,reason_for_stop,raw_enforcement_taken
0,1|2|3,2012-01-01,01:16:00,livingston and carpenter,39.949920,-82.975067,11.0,5.0,white,female,vehicular,True,False,False,arrest,True,Moving Violation,Arrest|Vehicle Search|Driver Search
1,4,2012-01-01,01:20:00,E 16th Ave and Waldeck Ave,40.001043,-83.006871,4.0,4.0,white,male,vehicular,False,False,True,warning,False,Moving Violation,Verbal Warning
2,5,2012-01-01,07:54:00,king ave and cannon dr,39.990708,-83.020528,NaN,NaN,black,male,vehicular,False,False,True,warning,False,Moving Violation,Verbal Warning
3,6,2012-01-01,10:00:00,James rd and Livingston ave,39.946876,-82.915567,9.0,2.0,black,male,vehicular,False,False,True,warning,False,Moving Violation,Verbal Warning
4,7,2012-01-01,10:50:00,N Wilson Road and Twin Creeks,39.979569,-83.102749,15.0,3.0,black,female,vehicular,False,False,True,warning,False,Moving Violation,Verbal Warning


In [0]:
my_train.shape,my_val.shape,test.shape

((79281, 18), (24126, 18), (24750, 18))

In [0]:
#Checking to make sure I grabbed all of the data
my_train.shape[0] + my_val.shape[0] + test.shape[0] == df.shape[0]

True

In [0]:
df.shape

(128157, 18)

## Regression or Classification?
This will be a binary classification

In [0]:
#Citation and arrest will be merged into a single value and be the 
my_train['outcome'].value_counts(normalize=True)

citation    0.555233
warning     0.405169
arrest      0.039598
Name: outcome, dtype: float64

## Evaluation Metric
I will be using roc/auc to evaluate. I might also use accuracy, since I believe I can improve greatly over baseline.

## Data Exploration / Cleaning

In [0]:
my_train.describe()

,lat,lng,precinct,zone
count,72577.000000,72577.000000,69423.000000,69423.000000
mean,39.991674,-82.985187,10.144131,3.223010
std,0.061422,0.076864,5.670301,1.472846
min,39.414070,-84.164396,1.000000,1.000000
25%,39.952456,-83.017007,5.000000,2.000000
50%,39.977232,-82.991951,10.000000,4.000000
75%,40.023498,-82.958907,16.000000,5.000000
max,40.602883,-80.715531,20.000000,5.000000


In [0]:
my_train.describe(exclude='number')

,raw_row_number,date,time,location,subject_race,subject_sex,type,arrest_made,citation_issued,warning_issued,outcome,search_conducted,reason_for_stop,raw_enforcement_taken
count,79281,79281,79163,79281,79281,79281,79281,79281,79281,79281,79246,79281,79281,79281
unique,79281,1096,1437,35986,5,2,1,2,2,2,3,2,4,120
top,89469,2012-02-03 00:00:00,16:00:00,70 and CHAMPION,white,male,vehicular,False,True,False,citation,False,Moving Violation,Traffic Citation
freq,1,162,247,645,40114,51984,79281,76143,46299,46480,44000,74350,59412,39778
first,NaN,2012-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2014-12-31 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
my_train.isnull().sum()

raw_row_number              0
date                        0
time                      118
location                    0
lat                      6704
lng                      6704
precinct                 9858
zone                     9858
subject_race                0
subject_sex                 0
type                        0
arrest_made                 0
citation_issued             0
warning_issued              0
outcome                    35
search_conducted            0
reason_for_stop             0
raw_enforcement_taken       0
dtype: int64

This is interesting. All of the entries where an outcome is missing, there were vehicle searches.
Still, there are only 35 instances. It's probably safe to drop all of the nan outcomes without losing any value

In [0]:
my_train[my_train['outcome'].isnull() == True]

,raw_row_number,date,time,location,lat,lng,precinct,zone,subject_race,subject_sex,type,arrest_made,citation_issued,warning_issued,outcome,search_conducted,reason_for_stop,raw_enforcement_taken
7196,8480,2012-03-09,20:10:00,BAIRSFORD DR and SARANAC DR,39.941613,-82.844087,14.0,2.0,black,female,vehicular,False,False,False,NaN,True,Investigatory,Vehicle Search
7443,8764|8765,2012-03-12,01:11:00,Easton Way and 270 s/b ramp,NaN,NaN,NaN,NaN,black,male,vehicular,False,False,False,NaN,True,Investigatory,Vehicle Search|Driver Search
8805,10360,2012-03-26,08:20:00,Fairwood and Forest,39.946118,-82.953945,11.0,5.0,black,male,vehicular,False,False,False,NaN,True,Moving Violation,Driver Search
9040,10627,2012-03-28,21:16:00,Interstate 70 and US Route 23,39.952785,-82.998751,11.0,5.0,white,male,vehicular,False,False,False,NaN,True,Investigatory,Vehicle Search
14086,16387,2012-05-22,18:55:00,Karl Rd and Satinwood Dr,40.084019,-82.974706,18.0,1.0,white,male,vehicular,False,False,False,NaN,True,Investigatory,Vehicle Search
15621,18124|18125,2012-06-10,21:45:00,Davis and Suulivant,39.953454,-83.020394,8.0,3.0,black,male,vehicular,False,False,False,NaN,True,Investigatory,Vehicle Search|Driver Search
18207,21152,2012-07-17,16:26:00,71 sb and 270 (north),39.980192,-82.856673,14.0,2.0,hispanic,male,vehicular,False,False,False,NaN,True,Moving Violation,Vehicle Search
20975,24439,2012-08-25,05:06:00,E. 161 and Tamarack Blvd,40.086852,-82.969602,18.0,1.0,white,male,vehicular,False,False,False,NaN,True,Investigatory,Vehicle Search
25923,30140,2012-11-08,05:10:00,Sullivant Ave and S Guilford Ave,39.950223,-83.037690,8.0,3.0,black,male,vehicular,False,False,False,NaN,True,Investigatory,Vehicle Search
26823,31185,2012-11-21,01:20:00,central and sullivant,39.950250,-83.036263,8.0,3.0,white,male,vehicular,False,False,False,NaN,True,Investigatory,Vehicle Search


In [0]:
df['location'].value_counts()

hamilton and 70                 758
i670wb and i71                  663
70 and CHAMPION                 647
70 and OHIO                     605
broad and mcnaughten            569
                               ... 
Morse Rd and Kensingwood Dr       1
E Broad St and Lancaster Ave      1
Gilbert St and E Sycamore St      1
alkire and forestwind             1
E. 5th Ave and N 5th St           1
Name: location, Length: 54338, dtype: int64

In [0]:
my_train['precinct'].value_counts(dropna=False
)

NaN     9858
16.0    7196
4.0     6655
3.0     5227
7.0     4270
12.0    4184
18.0    3924
9.0     3924
6.0     3693
17.0    3299
8.0     3228
14.0    3145
10.0    3110
11.0    2582
5.0     2511
1.0     2484
19.0    2431
2.0     2276
20.0    2240
13.0    1542
15.0    1502
Name: precinct, dtype: int64

- 'raw_row_number' just seems to count each individual action against a person. Three actions were taken against the first person, so that is raw_row_number 1,2, and 3.  That probably isn't useful, so I'll remove it


- 'location' is interesting in that is an intersection. I need to explore it more to see if it 'x and y' and also 'y and x' exist. I could strip out the ' and ' and separate the two crossroads into different columns.


- 'type' is a single value, so I can drop it


- 'arrest_made',  'citation_issued', and 'warning_issued' can be dropped because 'warning_issued' will be used as my binary classifier. I will need to invert the t/f and astype it to an integer.


- I need to do more exploration on 'search_conducted'. I'll likely drop that as data leakage since I feel like it would be highly correlated with action taken... After further exploration, the data is too dirty to deal with. There are multiple spellings, spelling errors, formatting differences. Also, this is covered in long/lat coordinates.


- 'raw_enforcement_taken' will be dropped as it includes my target

## Data Wrangling

In [0]:
def wrangle(X):
    
    #To prevent a warning.
    X = X.copy()
    
    # Pulling hour and minute from time column
    X['hour'] = pd.to_datetime(X['time']).dt.hour
    X['minute'] = pd.to_datetime(X['time']).dt.minute
    #X = X.drop(columns='time')
    
    
    #Dropping unecessary columns
    X = X.drop(columns=['raw_row_total','type','location'])
    
    #Dropping data leakage/time-traveling data
    X = X.drop(columns=['arrest_made','citation_issued','search_conducted',
                       'raw_enforcement_taken'] )
    
    
    #Creating separate columns if a row has these values missing.
    col_has_missing = ['time','lat','lng','precinct','zone']
    for column in col_has_missing:
        X[column+'_MISSING'] = x[column].isnull()
    
    return X
    
    